## LSTM with validation set

### Imports

In [81]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
import keras
import tensorflow as tf
from keras.layers import LSTM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Load data

In [82]:
df1 = pd.read_json("./data/Sarcasm_Headlines_Dataset.json", lines=True)
df2 = pd.read_json("./data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
# re-order attibute columns in df2
df2 = df2[['article_link','headline','is_sarcastic']]
df = pd.concat([df1, df2], axis=0)
df = df.drop(['article_link'], axis=1)
print(len(df))
df.head()

55328


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [83]:
df['len'] = df['headline'].apply(lambda x: len(x.split(" ")))
print(df['len'].mean())
print(min(df['len']))
print(max(df['len']))

9.953368999421631
2
151


### Split data in train, test and val
50% for train, 25% for test, 25% for validation.

In [84]:
train, test = train_test_split(df, test_size=0.25)
print(train.shape)
train, val = train_test_split(train, test_size=0.33333)

(41496, 3)


In [85]:
print('train:',train.shape)
print('test:',test.shape)
print('val:',val.shape)

train: (27664, 3)
test: (13832, 3)
val: (13832, 3)


### Tokenize
Fit the tokenizer only on the text of training data.
Then, we use that same tokenizer to transform the texts of train, val and test sets to sequences of integers.

It's possible to fit on the entire data. But it's probably a better idea to reserve a token for "unknown" words (oov_token=True), for the cases when you find new test data with words your model has never seen.

In [86]:
max_features = 10000 # max num words
maxlen = 25 # we could also try 151
embedding_size = 200

# create the tokenizer with the maximum number of words to keep, 
# based on word frequency. 
# Only the most common num_words-1 words will be kept.
tokenizer = Tokenizer(num_words=max_features, oov_token = True)

# fit the tokenizer on the headlines
tokenizer.fit_on_texts(list(train['headline']))

# Transforms each text in texts to a sequence of integers.
train_X = tokenizer.texts_to_sequences(train['headline'])
test_X = tokenizer.texts_to_sequences(test['headline'])
val_X = tokenizer.texts_to_sequences(val['headline'])

# transforms a list of num_samples sequences (lists of integers)
# into a 2D Numpy array of shape (num_samples, num_timesteps).
train_X = pad_sequences(train_X, maxlen = maxlen)
test_X = pad_sequences(test_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)

train_y = train['is_sarcastic']
test_y = test['is_sarcastic']
val_y = val['is_sarcastic']

Load glove embedding set, construct embedding matrix for words in word_index:


In [87]:
# load embeddings
EMBEDDING_FILE = './embeddings/glove.6B.200d.txt'

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

# first, build index mapping words in the embeddings set
# to their embedding vector
embeddings_index = {}
with open(EMBEDDING_FILE, encoding="utf8") as f:
    for line in f:
        word, coefs = get_coefs(*line.split(" "))
        embeddings_index[word] = coefs
            
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

# Random embedding vector for unknown words.
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
# prepare embedding matrix
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        # words not found in embedding index will be random
        embedding_matrix[i] = embedding_vector

C:\Users\elsas\Anaconda3\envs\TFM\lib\site-packages\IPython\core\interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


### LSTM Model
Model Parameters:

- **Activation Function**: I have used ReLU as the activation function. ReLU is a non-linear activation function, which helps complex relationships in the data to be captured by the model.

- **Optimiser**: We use adam optimiser, which is an adaptive learning rate optimiser.

- **Loss function**: We will train a network to output a probability over the 2 classes using Sigmoid Loss.

In [93]:
# Create model structure

model = Sequential()
model.add(Embedding(max_features, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives()])

### save the best model and early stopping
To prevent the model from overfitting I have enabled early stopping.

Early stopping is a method that allows us to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold out/validation dataset.


In [94]:
# Save the model after every epoch.
saveBestModel = keras.callbacks.ModelCheckpoint('./model/best_model.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# Stop training when a monitored quantity has stopped improving.
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [95]:
# Fit the model
batch_size = 100
epochs = 25
model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_X, val_y), callbacks=[saveBestModel, earlyStopping])

C:\Users\elsas\Anaconda3\envs\TFM\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 27664 samples, validate on 13832 samples
Epoch 1/25
27664/27664 [==============================] - 57s 2ms/step - loss: 0.5752 - accuracy: 0.6787 - precision_4: 0.7091 - recall_4: 0.5004 - true_positives_4: 6310.0000 - val_loss: 0.3429 - val_accuracy: 0.8533 - val_precision_4: 0.8458 - val_recall_4: 0.8340 - val_true_positives_4: 5319.0000
Epoch 2/25


C:\Users\elsas\Anaconda3\envs\TFM\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


27664/27664 [==============================] - 54s 2ms/step - loss: 0.3133 - accuracy: 0.8909 - precision_4: 0.8928 - recall_4: 0.8646 - true_positives_4: 10902.0000 - val_loss: 0.2785 - val_accuracy: 0.8905 - val_precision_4: 0.8761 - val_recall_4: 0.8882 - val_true_positives_4: 5665.0000
Epoch 3/25
27664/27664 [==============================] - 54s 2ms/step - loss: 0.2010 - accuracy: 0.9374 - precision_4: 0.9451 - recall_4: 0.9159 - true_positives_4: 11549.0000 - val_loss: 0.2470 - val_accuracy: 0.9041 - val_precision_4: 0.9223 - val_recall_4: 0.8648 - val_true_positives_4: 5516.0000
Epoch 4/25
27664/27664 [==============================] - 54s 2ms/step - loss: 0.1358 - accuracy: 0.9611 - precision_4: 0.9692 - recall_4: 0.9446 - true_positives_4: 11911.0000 - val_loss: 0.2657 - val_accuracy: 0.9137 - val_precision_4: 0.9226 - val_recall_4: 0.8873 - val_true_positives_4: 5659.0000
Epoch 5/25
27664/27664 [==============================] - 54s 2ms/step - loss: 0.1014 - accuracy: 0.9734 

### Evaluate model results with test data

In [96]:
loss, accuracy, f1_score, precision, recall = model.evaluate(test_X, test_y, batch_size=batch_size)

13832/13832 [==============================] - 6s 450us/step


#### Loss, Accuracy, Precision, Recall and F1

In [97]:
print(loss)
print(accuracy)
print(f1_score)
print(precision)
print(recall)

0.3471229235889975
0.9209080338478088
0.9328848123550415
0.8924646973609924
5685.0


Stops in the Epoch 6 out of 25, this is thanks to the validation set, thta prevents us to overfit the model.